In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My Drive/time-series-hourly-energy-usage

/content/drive/My Drive/time-series-hourly-energy-usage


In [3]:
import pickle
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from utils import EnergyDemandDataset, AverageMeter, save_checkpoint

In [4]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, output_size=1):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        self.lstm = torch.nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.linear = torch.nn.Linear(in_features=self.hidden_size, out_features=self.output_size)
    
    def forward(self, x):
        h, _ = self.lstm(x)
        x = self.linear(h[:,-1,:])
        return x

## Determine the baseline MSE

In [ ]:
val_rmse = AverageMeter()
MEAN = 18251.291178385418
STD = 3475.1125327833797
val_loader = DataLoader(EnergyDemandDataset('val', 24*7, transform = 'standard'), batch_size = 16, shuffle = False)
train_set_mean = np.mean(EnergyDemandDataset('train', 1).data)
loss = nn.MSELoss()
for i, (sequence, target) in enumerate(val_loader):
      batch_size = target.shape[0]
      val_rmse.update(torch.sqrt(loss(torch.ones(batch_size,24)*train_set_mean*STD + MEAN, 
                          target.squeeze(1)*STD + MEAN)), batch_size)
print(val_rmse.avg.item())

3006.132080078125


# Train Simple LSTM

In [5]:
HIDDEN_SIZE = 16
NUM_LAYERS = 5
NUM_EPOCHS = 50
LEARNING_RATE = 0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleLSTM(input_size = 1, 
                   hidden_size=HIDDEN_SIZE, 
                   num_layers=NUM_LAYERS, 
                   output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [7]:
train_loader = DataLoader(EnergyDemandDataset('train',  2500, transform = 'standard'), batch_size = 16, shuffle = True)
val_loader = DataLoader(EnergyDemandDataset('val', 2500, transform = 'standard'), batch_size = 16, shuffle = False)
# sequence, target = next(iter(val_loader))

In [8]:
min_rmse = 1e10
MEAN = 18251.291178385418
STD = 3475.1125327833797
for epoch in range(NUM_EPOCHS):
    if (epoch+1) % 10 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] /= 10
    model.train()
    print('Epoch [{}/{}]'.format(epoch+1, NUM_EPOCHS))
    # Training Loop
    for i, (sequence, target) in enumerate(train_loader):
        sequence = sequence.unsqueeze(2).to(device)
        output = model(sequence).cpu()
        loss = criterion(output, target)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 500 == 0:
          print('Loss: {}'.format(loss.item()))
    # Validation Loop
    val_rmse = AverageMeter()
    val_loss = nn.MSELoss()
    model.eval()
    with torch.no_grad():
        for i, (sequence, target) in enumerate(val_loader):
            batch_size = target.shape[0]
            look_back = sequence.shape[-1]
            sequence = sequence.unsqueeze(2).to(device)
            for step in range(target.shape[-1]):
                output = model(sequence)
                sequence = torch.cat((sequence, output.unsqueeze(2)), 1)
            val_rmse.update(torch.sqrt(val_loss((sequence[:,-24:].squeeze(2)*STD + MEAN).cpu(), 
                          target.squeeze(1)*STD + MEAN)), batch_size)
    total_val_rmse = val_rmse.avg.item()
    print('Validation RMSE: {}'.format(total_val_rmse))
    is_best = total_val_rmse < min_rmse
    if is_best:
        min_rmse = total_val_rmse
    save_checkpoint(model.state_dict(), is_best)

Epoch [1/50]
Loss: 0.7497952580451965
Loss: 1.6170940399169922
Loss: 0.5720080137252808
Loss: 1.1965316534042358


RuntimeError: ignored

In [8]:
x = np.arange(1000000)

In [12]:
look_back = 40
i = 1
x[0+i*24:look_back+i*24]

array([24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63])

In [ ]:
val_mse = AverageMeter()
loss = nn.MSELoss()
model.eval()
with torch.no_grad():
  for i, (sequence, target) in enumerate(val_loader):
      batch_size = target.shape[0]
      look_back = sequence.shape[-1]
      sequence = sequence.unsqueeze(2).to(device)
      for step in range(target.shape[-1]):
          output = model(sequence[:,-look_back:,:])
          sequence = torch.cat((sequence, output.unsqueeze(2)), 1)
      val_mse.update(loss(sequence[:,-24:].squeeze(2).cpu(), target.squeeze(1)), batch_size)
    